In [70]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st

import re
import nltk
from nltk.tag import hmm
from nltk.corpus import brown

import nltk.lm as lm
from nltk.util import ngrams as nltk_ngrams

from collections import Counter

In [5]:
data = pd.read_csv('../data/sotnikov.ad_hw7.csv', header=None)

In [6]:
data.head()

,0
0,1
1,0
2,1
3,1
4,0


In [7]:
data.describe()

,0
count,34.000000
mean,0.529412
std,0.506640
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [16]:
x = [a for b in data.values for a in b]

In [17]:
x

[1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1]

In [62]:
len(x)

34

#### 1) Оценка p(X_{i+1}= 0|X_{i}=0) методом максимального правдоподобия

In [57]:
def ngrams_and_prefix_counts(tokens, n_max):
    # словарь n-грамм и их частот
    ngrams_counts = {}
    # словарь n-граммных префиксов и их частот
    prefix_counts = {}
    
    n = len(tokens)
    for i in range(n_max):
        ngrams_counts[i + 1] = Counter([tuple(tokens[j : j + i + 1]) for j in range(n - i)])
        prefix_counts[i + 1] = Counter([tuple(tokens[j : j + i] + ['*']) for j in range(n - i)])

    return ngrams_counts, prefix_counts

In [56]:
def bigram_probas(ngram_counts, prefix_counts):
    p2 = {}
    for w in ngram_counts[2]:
        pre_w = tuple([w[0]] + ['*'])
        p2[u'{1}|{0}'.format(*w)] = ngram_counts[2][w] / prefix_counts[2][pre_w]
    return p2

In [66]:
a, b = ngrams_and_prefix_counts(x, 3)

In [67]:
a, b

({1: Counter({(1,): 18, (0,): 16}),
  2: Counter({(1, 0): 14, (0, 1): 14, (1, 1): 3, (0, 0): 2}),
  3: Counter({(1, 0, 1): 12,
           (0, 1, 1): 3,
           (1, 1, 0): 3,
           (0, 1, 0): 10,
           (1, 0, 0): 2,
           (0, 0, 1): 2})},
 {1: Counter({('*',): 34}),
  2: Counter({(1, '*'): 17, (0, '*'): 16}),
  3: Counter({(1, 0, '*'): 14,
           (0, 1, '*'): 13,
           (1, 1, '*'): 3,
           (0, 0, '*'): 2})})

In [68]:
p2 = bigram_probas(a, b)
p2

{'0|1': 0.8235294117647058,
 '1|0': 0.875,
 '1|1': 0.17647058823529413,
 '0|0': 0.125}

#### 2) Отвергается ли нулевая гипотеза p(X_{i+1}= 0|X_{i}=0)=0.9?

#### 3) Какова вероятность встретить Xi=2 с применением сглаживания Лапласа?

In [78]:
x = [str(a) for a in x]

In [79]:
n1 = list(nltk_ngrams(x, 1))
n2 = list(nltk_ngrams(x, 2))
n3 = list(nltk_ngrams(x, 3))

In [80]:
n3

[('1', '0', '1'),
 ('0', '1', '1'),
 ('1', '1', '0'),
 ('1', '0', '1'),
 ('0', '1', '0'),
 ('1', '0', '1'),
 ('0', '1', '0'),
 ('1', '0', '0'),
 ('0', '0', '1'),
 ('0', '1', '0'),
 ('1', '0', '1'),
 ('0', '1', '0'),
 ('1', '0', '1'),
 ('0', '1', '0'),
 ('1', '0', '1'),
 ('0', '1', '1'),
 ('1', '1', '0'),
 ('1', '0', '1'),
 ('0', '1', '0'),
 ('1', '0', '1'),
 ('0', '1', '1'),
 ('1', '1', '0'),
 ('1', '0', '1'),
 ('0', '1', '0'),
 ('1', '0', '1'),
 ('0', '1', '0'),
 ('1', '0', '1'),
 ('0', '1', '0'),
 ('1', '0', '1'),
 ('0', '1', '0'),
 ('1', '0', '0'),
 ('0', '0', '1')]

In [82]:
laplace = lm.Laplace(order=3)
laplace.fit([n1] + [n2] + [n3], vocabulary_text=['0','1'])

In [87]:
round(laplace.score('5'), 3)

0.027